(ch_vector_add)=
# 向量加法

编写程序：对两个 `n` 维向量 `a` 和 `b` 求和。这在 NumPy 中很简单，可以用 `c = a + b` 来做。

## NumPy 实现

In [1]:
import numpy as np

np.random.seed(0)
n = 100
a = np.random.normal(size=n).astype(np.float32)
b = np.random.normal(size=n).astype(np.float32)
c = a + b

这里，创建了两个长度为 100 的随机向量，并对它们进行元素求和。注意，NumPy 在默认情况下使用 64 位浮点数或 64 位整数，这与深度学习中通常使用的 32 位浮点数不同，因此需要显式转换数据类型。

虽然可以在 NumPy 中使用内置的 `+` 算子（operator）来实现元素级的加法，但这里尝试仅使用标量算子来实现它。它将帮助理解 TVM 的实现。下面的函数使用 `for` 循环迭代向量中的每个元素，然后每次使用标量（scalar） `+` 算子将两个元素相加。

In [2]:
def vector_add(a, b, c):
    n = len(a)
    for i in range(n):
        c[i] = a[i] + b[i]

d = np.empty(shape=n, dtype=np.float32)
vector_add(a, b, d)
np.testing.assert_array_equal(c, d)

在接下来的章节中，将经常创建两个随机的 `ndarray` 和另一个空的 `ndarray` 来存储结果，保存这个例子以便将来重用它。

In [3]:
# 保存到 d2ltvm 包
def get_abc(shape, constructor=None):
    """Return random a, b and empty c with the same shape.
    """
    np.random.seed(0)
    a = np.random.normal(size=shape).astype(np.float32)
    b = np.random.normal(size=shape).astype(np.float32)
    c = np.empty_like(a)
    if constructor:
        a, b, c = [constructor(x) for x in (a, b, c)]
    return a, b, c

```{attention}
固定 random seed，这样将总是得到相同的结果，以方便 NumPy、TVM 和其他类型之间的比较。此外，它还接受可选的 `constructor` 来将数据转换为不同的格式。
```

## 定义 TVM 计算

为了加载 TVM 环境，需要：

In [4]:
from tvm_book.tvm.env import set_tvm # 设置 TVM 环境
tvm_root = "/media/pc/data/4tb/lxw/books/tvm"
set_tvm(tvm_root)

现在开始在 TVM 中实现 {func}`vector_add`。TVM 的实现有两点不同：

1. 不需要编写完整的函数，只需要指定每个元素的输出，即 `c[i]`，被计算
2. TVM 是符号的，通过指定形状来创建符号变量，并定义程序如何计算

在下面的程序中，首先通过 {func}`tvm.te.placeholder` 来指定两个输入的占位符 `A` 和 `B` 的形状 `(n,)`。`A` 和 `B` 都是 `Tensor` 对象，可以稍后提供数据。给它们赋值名称，这样以后就可以打印易于阅读的程序。

接下来，定义如何通过 {func}`tvm.compute` 计算输出 `C`。它接受两个参数：输出形状和通过给定索引来计算每个元素的函数。由于输出是向量，它的元素按整数索引。在 {func}`tvm.compute` 中定义的 lambda 函数接受单个参数 `i`，并返回 `c[i]`，它与 {func}`vector_add` 中定义的 `c[i] = a[i] + b[i]` 相同。不同之处在于，不用编写 for 循环，它将在稍后由 TVM 填充。

In [5]:
import tvm
from tvm import te  # te 代表张量表达式（tensor expression）


def vector_add(n): # 保存到 d2ltvm
    """TVM expression for vector add"""
    A = te.placeholder((n,), name='a')
    B = te.placeholder((n,), name='b')
    C = te.compute(A.shape, lambda i: A[i] + B[i], name='c')
    return A, B, C


A, B, C = vector_add(n)
type(A), type(C)

(tvm.te.tensor.Tensor, tvm.te.tensor.Tensor)

可以看到 `A`、`B` 和 `C` 都是 {class}`~tvm.te.tensor.Tensor` 对象，它可以被视为 NumPy ndarray 的符号版本。可以访问变量的属性，比如数据类型和形状。但这些值目前还没有具体值。

In [6]:
(A.dtype, A.shape), (C.dtype, C.shape)

(('float32', [100]), ('float32', [100]))

生成张量对象的算子可以被 `.op` 访问。

In [7]:
type(A.op), type(C.op)

(tvm.te.tensor.PlaceholderOp, tvm.te.tensor.ComputeOp)

可以看到 `A` 和 `C` 的算子类型是不同的，但它们共享同一个基类 {class}`~tvm.te.tensor.Operation`，这个基类表示生成张量对象的算子。

In [8]:
A.op.__class__.__bases__[0], C.op.__class__.__bases__[0]

(tvm.te.tensor.Operation, tvm.te.tensor.BaseComputeOp)

In [9]:
issubclass(tvm.te.tensor.BaseComputeOp, tvm.te.tensor.Operation)

True

## 创建 Schedule

为了运行计算，需要指定如何执行程序，例如，访问数据的顺序以及如何进行多线程并行化。
这样的执行计划被称为 **调度** （schedule）。`C` 是输出张量，可以在它的算子上创建默认调度，并打印伪代码。

In [10]:
s = te.create_schedule(C.op)
s

schedule(0x56108a67af20)

调度由几个阶段（Stage）组成。每个阶段对应于描述它是如何调度的算子。可以通过 `s[C]` 或 `s[C.op]` 进入特定的阶段。

In [11]:
type(s), type(s[C])

(tvm.te.schedule.Schedule, tvm.te.schedule.Stage)

稍后将看到如何更改执行计划，以便更好地利用硬件资源来提高其效率。下面通过打印 C-like 的伪代码来查看默认的执行计划。

In [12]:
m = tvm.lower(s, [A, B, C], simple_mode=True)
print(m)

@main = primfn(a_1: handle, b_1: handle, c_1: handle) -> ()
  attr = {"from_legacy_te_schedule": True, "global_symbol": "main", "tir.noalias": True}
  buffers = {a: Buffer(a_2: Pointer(float32), float32, [100], []),
             b: Buffer(b_2: Pointer(float32), float32, [100], []),
             c: Buffer(c_2: Pointer(float32), float32, [100], [])}
  buffer_map = {a_1: a, b_1: b, c_1: c}
  preflattened_buffer_map = {a_1: a_3: Buffer(a_2, float32, [100], []), b_1: b_3: Buffer(b_2, float32, [100], []), c_1: c_3: Buffer(c_2, float32, [100], [])} {
  for (i: int32, 0, 100) {
    c[i] = (a[i] + b[i])
  }
}




{func}`tvm.lower` 方法接受 schedule、输入、输出张量。`simple_mode=True` 将以简单而紧凑的方式打印程序。注意，程序已经根据输出形状添加了适当的 for 循环。总的来说，它非常类似于前面的函数 `vector_add`。

可以看到 TVM 将计算和调度分开。计算定义了如何计算结果，无论在什么硬件平台上运行程序，结果都不会改变。另一方面，有效的调度通常依赖于硬件，但是更改调度不会影响其正确性。TVM 从 Halide {cite:p}`Ragan-Kelley.Barnes.Adams.ea.2013` 那里继承了计算与进度分离的想法。

## 计算并执行

一旦定义了计算和调度，就可以使用 {func}`tvm.build` 将它们编译成可执行模块。它接受与 {func}`tvm.lower` 相同的参数。实际上，它首先调用了 {func}`tvm.lower` 生成中间表示程序，然后编译成机器码。即

```python
mod = tvm.build(s, [A, B, C])
mod
```

或者，直接把中间表示模块编译为结果模块：

In [13]:
mod = tvm.build(m)
type(mod)

tvm.driver.build_module.OperatorModule

它返回可执行模块对象。可以输入 `A`，`B` 和 `C` 的数据来运行它。张量数据必须是 {class}`tvm.runtime.ndarray.NDArray` 对象。最简单的方法是首先创建 NumPy ndarray 对象，然后通过 {func}`tvm.nd.array` 将它们转换为 TVM ndarray 对象。可以通过 {meth}`.numpy` 方法将它们转换回 NumPy。

In [14]:
x = np.ones(2)
y = tvm.nd.array(x)
type(y), y.numpy()

(tvm.runtime.ndarray.NDArray, array([1., 1.]))

构造数据并将它们作为 TVM ndarray 返回：

In [15]:
a, b, c = get_abc(100, tvm.nd.array)

进行计算，并验证结果。

In [16]:
mod(a, b, c)
np.testing.assert_array_equal(a.numpy() + b.numpy(), c.numpy())

## 参数约束

记住，当声明 `A` 和 `B` 时，将两个输入都指定长度为 100 的向量。

In [17]:
A.shape, B.shape, C.shape

([100], [100], [100])

TVM 将检查输入形状是否满足此规格。

In [18]:
try:
    a, b, c = get_abc(200, tvm.nd.array)
    mod(a, b, c)
except tvm.TVMError as e:
    print(e)

Traceback (most recent call last):
  1: TVMFuncCall
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::WrapPackedFunc(int (*)(TVMValue*, int*, int, TVMValue*, int*, void*), tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  File "/media/pc/data/4tb/lxw/books/tvm/src/runtime/library_module.cc", line 80
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: ret == 0 (-1 vs. 0) : Assert fail: (100 == int32(arg.a.shape[0])), Argument arg.a.shape[0] has an unsatisfied constraint: (100 == int32(arg.a.shape[0]))


TVM 默认数据类型为 `float32`。

In [19]:
A.dtype, B.dtype, C.dtype

('float32', 'float32', 'float32')

如果输入的数据类型不同，则会出现错误。


In [20]:
try:
    a, b, c = get_abc(100, tvm.nd.array)
    a = tvm.nd.array(a.numpy().astype('float64'))
    mod(a, b, c)
except tvm.TVMError as e:
    print(e)

Traceback (most recent call last):
  1: TVMFuncCall
  0: tvm::runtime::PackedFuncObj::Extractor<tvm::runtime::PackedFuncSubObj<tvm::runtime::WrapPackedFunc(int (*)(TVMValue*, int*, int, TVMValue*, int*, void*), tvm::runtime::ObjectPtr<tvm::runtime::Object> const&)::{lambda(tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)#1}> >::Call(tvm::runtime::PackedFuncObj const*, tvm::runtime::TVMArgs, tvm::runtime::TVMRetValue*)
  File "/media/pc/data/4tb/lxw/books/tvm/src/runtime/library_module.cc", line 80
TVMError: 
---------------------------------------------------------------
An error occurred during the execution of TVM.
For more information, please see: https://tvm.apache.org/docs/errors.html
---------------------------------------------------------------

  Check failed: ret == 0 (-1 vs. 0) : Assert fail: (((tir.tvm_struct_get(arg.a, 0, 5) == (uint8)2) && (tir.tvm_struct_get(arg.a, 0, 6) == (uint8)32)) && (tir.tvm_struct_get(arg.a, 0, 7) == (uint16)1)), arg.a.dtype is expected to be fl

## 保存和加载模块

编译好的模块可以保存到磁盘中：

In [21]:
mod_fname = 'vector-add.tar'
mod.export_library(mod_fname)

然后再加载回来。

In [22]:
loaded_mod = tvm.runtime.load_module(mod_fname)

验证结果：

In [23]:
a, b, c = get_abc(100, tvm.nd.array)
loaded_mod(a, b, c)
np.testing.assert_array_equal(a.numpy() + b.numpy(), c.numpy())

## 小结

使用 TVM 实现算子有三个步骤：

1. 通过指定输入和输出形状以及如何计算每个输出元素来声明计算。
2. 创建调度，（希望）充分利用机器资源。
3. 编译到硬件目标。

此外，可以将编译后的模块保存到磁盘中，以便稍后再将其加载回来。